## Responses APIとは？

Responses APIは、OpenAIモデルと対話するための新しい方法で、従来のAPIよりもシンプルで柔軟性に優れた設計となっています。複数のツールを使用し、マルチターン会話を処理し、（テキストだけでなく）さまざまなタイプのデータを扱う高度なAIアプリケーションを簡単に構築できます。

主にテキスト用に構築されたChat Completionsや、多くのセットアップが必要なAssistants APIなどの古いAPIとは異なり、Responses APIは以下の機能のために一から構築されています：

- シームレスなマルチターン対話（単一のAPI呼び出しで複数のステップにわたって会話を継続）
- 強力なホストツールへの簡単なアクセス（ファイル検索、ウェブ検索、コードインタープリターなど）
- モデルに送信するコンテキストの細かい制御

AIモデルが複雑で長時間実行される推論をより適切に処理できるようになるにつれて、開発者は非同期かつステートフルなAPIを必要としています。Responses APIは、これらのニーズを満たすように設計されています。

このガイドでは、Responses APIが提供する新機能の一部と、始めるのに役立つ実用的な例をご紹介します。

## 基本事項
設計上、表面的には、Responses APIはCompletions APIと非常に似ています。

In [1]:
from openai import OpenAI
import os
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [5]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="tell me a joke",
)


In [14]:
print(response.output[0].content[0].text)

Why did the scarecrow win an award?

Because he was outstanding in his field!


Response APIの主要な特徴の一つは、ステートフルであることです。これは、会話の状態を自分で管理する必要がなく、APIが代わりに処理してくれることを意味します。例えば、いつでもレスポンスを取得でき、完全な会話履歴が含まれます。

In [15]:
fetched_response = client.responses.retrieve(
response_id=response.id)

print(fetched_response.output[0].content[0].text)


Why did the scarecrow win an award?

Because he was outstanding in his field!


前の回答を参照して会話を続けることができます。

In [11]:
response_two = client.responses.create(
    model="gpt-4o-mini",
    input="tell me another",
    previous_response_id=response.id
)


In [12]:
print(response_two.output[0].content[0].text)

Why don't skeletons fight each other?

They don't have the guts!


もちろん、コンテキストを自分で管理することもできます。しかし、OpenAIがコンテキストを管理してくれることの利点の一つは、任意の時点でレスポンスを分岐させ、その時点から会話を続けることができることです。

In [13]:
response_two_forked = client.responses.create(
    model="gpt-4o-mini",
    input="I didn't like that joke, tell me another and tell me the difference between the two jokes",
    previous_response_id=response.id # Forking and continuing from the first response
)

output_text = response_two_forked.output[0].content[0].text
print(output_text)

Sure! Here’s another joke:

Why don’t scientists trust atoms?

Because they make up everything!

**Difference:** The first joke plays on a pun involving "outstanding" in a literal sense versus being exceptional, while the second joke relies on a play on words about atoms "making up" matter versus fabricating stories. Each joke uses wordplay, but they target different concepts (farming vs. science).


## ホストされたツール

Responses APIのもう一つの利点は、`file_search`や`web_search`のようなホストされたツールのサポートが追加されることです。手動でツールを呼び出す代わりに、単純にツールを渡すだけで、APIがどのツールを使用するかを決定し、それを使用します。

以下は、`web_search`ツールを使用してウェブ検索結果をレスポンスに組み込む例です。

In [16]:
response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input="What's the latest news about AI?",
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [17]:
import json
print(json.dumps(response.output, default=lambda o: o.__dict__, indent=2))

[
  {
    "id": "ws_67bd64fe91f081919bec069ad65797f1",
    "status": "completed",
    "type": "web_search_call"
  },
  {
    "id": "msg_67bd6502568c8191a2cbb154fa3fbf4c",
    "content": [
      {
        "annotations": [
          {
            "index": null,
            "title": "Huawei improves AI chip production in boost for China's tech goals",
            "type": "url_citation",
            "url": "https://www.ft.com/content/f46b7f6d-62ed-4b64-8ad7-2417e5ab34f6?utm_source=chatgpt.com"
          },
          {
            "index": null,
            "title": "Apple cheers Trump with $500bn US investment plan; more losses on Wall Street - as it happened",
            "type": "url_citation",
            "url": "https://www.theguardian.com/business/live/2025/feb/24/euro-hits-one-month-high-german-election-result-stock-markets-dax-bank-of-england-business-live-news?utm_source=chatgpt.com"
          },
          {
            "index": null,
            "title": "Microsoft axes data cente

## マルチモーダル、ツール拡張会話

Responses APIは、テキスト、画像、音声のモダリティをネイティブにサポートしています。
すべてを組み合わせることで、Responses APIを通じて1回のAPI呼び出しで完全にマルチモーダルでツール拡張された対話を構築できます。

In [19]:
import base64

from IPython.display import Image, display

# Display the image from the provided URL
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Cat_August_2010-4.jpg/2880px-Cat_August_2010-4.jpg"
display(Image(url=url, width=400))

response_multimodal = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": 
                 "Come up with keywords related to the image, and search on the web using the search tool for any news related to the keywords"
                 ", summarize the findings and cite the sources."},
                {"type": "input_image", "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Cat_August_2010-4.jpg/2880px-Cat_August_2010-4.jpg"}
            ]
        }
    ],
    tools=[
        {"type": "web_search"}
    ]
)


In [22]:
import json
print(json.dumps(response_multimodal.__dict__, default=lambda o: o.__dict__, indent=4))

{
    "id": "resp_67bd65392a088191a3b802a61f4fba14",
    "created_at": 1740465465.0,
    "error": null,
    "metadata": {},
    "model": "gpt-4o-2024-08-06",
    "object": "response",
    "output": [
        {
            "id": "msg_67bd653ab9cc81918db973f0c1af9fbb",
            "content": [
                {
                    "annotations": [],
                    "text": "Based on the image of a cat, some relevant keywords could be:\n\n- Cat\n- Feline\n- Pet\n- Animal care\n- Cat behavior\n\nI'll search for recent news related to these keywords.",
                    "type": "output_text",
                    "logprobs": null
                }
            ],
            "role": "assistant",
            "type": "message"
        },
        {
            "id": "ws_67bd653c7a548191af86757fbbca96e1",
            "status": "completed",
            "type": "web_search_call"
        },
        {
            "id": "msg_67bd653f34fc8191989241b2659fd1b5",
            "content": [
           

上記の例では、Chat Completions APIを使用した場合に必要となる複数回のやり取りではなく、1回のAPI呼び出しで`web_search`ツールを使用して画像に関連するニュースをウェブ検索することができました。

Responses APIを使用すると
🔥 単一のAPI呼び出しで以下を処理できます：

✅ マルチモーダル入力を使用して指定された画像を分析する

✅ `web_search`ホストツールを介してWeb検索を実行する

✅ 結果を要約する

対照的に、Chat Completions APIでは複数のステップが必要で、それぞれAPIへのラウンドトリップが必要です：

1️⃣ 画像をアップロードして分析を取得 → 1回のリクエスト

2️⃣ 情報を抽出し、外部Web検索を呼び出し → 手動ステップ + ツール実行

3️⃣ 要約のためにツール結果を再送信 → 別のリクエスト

以下の図で並列の視覚的比較をご覧ください！

![Responses vs Completions](../../images/comparisons.png)

Responses APIをお試しいただき、コードを簡素化し、複雑でマルチモーダルなツール拡張インタラクションの構築を容易にする方法をご確認いただけることを大変楽しみにしています！